In [ ]:
# UPDATED to remove files with no FR_component (from horizontal splits)
# But in wrong language

In [15]:
# FASTTEXT LANGUAGE detection 
import fasttext as ft

# Load the pretrained model
ft_model = ft.load_model("lid.176.ftz")

def fasttext_language_predict(text, model = ft_model):

  text = text.replace('\n', " ")
  prediction = model.predict([text])

  return prediction

In [28]:
def fix_languages():
    # This function will correct the languages from the .txt files 
    # It switches the NL_ and FR_ .txt files when needed
    #
    # It also checks for incorrect split documents 
    # It removes their .txt files from the /processed/data folder
    # and copies these .pdfs to /unprocessed_data/removed_from_processed
    #
    # To be executed after the split and process_docs_cloud

    import os
    from os import listdir
    import time
    import sys
    import tempfile
    import shutil

    erroruous_split_files=[]
    switched_docs=[]
    not_french=0

    input_language='NL'
    opposite_language='FR'

    input_path=os.path.join('../processed_data',input_language)
    opposite_path=os.path.join('../processed_data',opposite_language)

    files_in_path=listdir(input_path)
    for file in files_in_path:
        filename=os.path.join(input_path,file)
 
        if 'NL_' in file:
            document_id=file[3:]
        else:
            document_id=file[:]

        with open(filename,encoding="utf-8") as f:
            text = f.read() 

        l_text=len(text) 

        db_check_input_language_1_half=fasttext_language_predict(text[:int(l_text/2)], model = ft_model)[0][0][0][-2:] 
        db_check_input_language_2_half=fasttext_language_predict(text[int(l_text/2):], model = ft_model)[0][0][0][-2:] 

        # Check if language detection is consistent throughout text
        if db_check_input_language_1_half==db_check_input_language_2_half:           

            detected_language=fasttext_language_predict(text, model = ft_model)[0][0][0][-2:]  

            # Check if language detected differs from folder language
            if detected_language.upper()!=input_language:
                
                try:
                    with open (f'{opposite_path}/{opposite_language}_{document_id}',encoding="utf-8") as f:
                        opposite_text=f.read()
                        
                        opp_detected_language=fasttext_language_predict(opposite_text, model = ft_model)[0][0][0][-2:]  

                except FileNotFoundError:

                    try: 
                        with open (f'{opposite_path}/{document_id}',encoding="utf-8") as f:
                            opposite_text=f.read()                        
                            opp_detected_language=fasttext_language_predict(opposite_text, model = ft_model)[0][0][0][-2:]  

                    except FileNotFoundError:
                        opp_detected_language='NOTFOUND'

                if opp_detected_language!='NOTFOUND':

                    # Check if other file actually differs in language
                    if detected_language!=opp_detected_language:

                        # Write the text to the opposite language folder

                        with open (f'{opposite_path}/{opposite_language}_{document_id}','w',encoding="utf-8") as fp:
                            fp.write(text)

                        # Write the opposite text to this language folder

                        with open (f'{input_path}/{input_language}_{document_id}','w',encoding="utf-8") as fp:
                            fp.write(opposite_text)

                        print (f"Switched document {document_id}",end='\n')
                        switched_docs.append(document_id)
                    else:
                        # THESE FILES WILL BE REMOVED and added back to UNPROCESSED

                        erroruous_split_files.append(document_id)
                        filename=f'{input_path}/{input_language}_{document_id}'
                        if os.path.exists(filename):
                            os.remove(filename)
                        filename=f'{opposite_path}/{opposite_language}_{document_id}'
                        if os.path.exists(filename):
                            os.remove(filename)
                        
                        # Move these files back to unprocessed_data folder
                        path=os.path.join('..','data')
                        import glob
                        import shutil
                        document_id=document_id[:-4]

                        for f in glob.glob(f'{path}/**', recursive=True):
                            
                            if f'{document_id}.pdf' in f:
                                print(f)

                                src_path=f
                                dest_path=os.path.join('..','unprocessed_data')    
                                dest_path=os.path.join(dest_path,'removed_from_processed')        

                                if not os.path.exists(dest_path):
                                    os.mkdir(dest_path)
                                    
                                dest_file=os.path.join(dest_path,document_id)
                                dest_file=f'{dest_file}.pdf'
                                
                                print (f"Notice: Removed document {document_id}",end='\n')
                                shutil.copy(src_path, dest_file)
                else:
                    # NOT FOUND REMOVE THIS FILE
 
                    os.remove(f'{filename}')    
                    print (f'{input_path}/{document_id} Removed - not found in other language')
                    not_french+=1

        """
        else:
            # THESE FILES WILL BE REMOVED and added back to UNPROCESSED

            erroruous_split_files.append(document_id)
            filename=f'{input_path}/{input_language}_{document_id}'
            if os.path.exists(filename):
                os.remove(filename)
            filename=f'{opposite_path}/{opposite_language}_{document_id}'
            if os.path.exists(filename):
                os.remove(filename)
            
            # Move these files back to unprocessed_data folder
            path=os.path.join('..','data')
            import glob
            import shutil
            document_id=document_id[:-4]

            for f in glob.glob(f'{path}/**', recursive=True):
                
                if f'{document_id}.pdf' in f:
                    print(f)

                    src_path=f
                    dest_path=os.path.join('..','unprocessed_data')    
                    dest_path=os.path.join(dest_path,'removed_from_processed')        

                    if not os.path.exists(dest_path):
                        os.mkdir(dest_path)
                        
                    dest_file=os.path.join(dest_path,document_id)
                    dest_file=f'{dest_file}.pdf'
                    
                    print (f"Notice: Removed document {document_id}",end='\n')
                    shutil.copy(src_path, dest_file)
                    """
    
    print (f'\nSwitched documents : {len(switched_docs)}')
    print (f'\nFrench docs with no counterpart : {not_french}')
    print (f'Error documents : {len(erroruous_split_files)} files moved back to unprocessed')

In [29]:
fix_languages()


Switched documents : 0

French docs with no counterpart : 0
Error documents : 0 files moved back to unprocessed
